In [1]:
import boto3
import re
import urllib.request as req

In [105]:
def get_public_ip():
    data = str(req.urlopen('http://checkip.dyndns.com/').read())
    
    return re.compile(r'Address: (\d+.\d+.\d+.\d+)').search(data).group(1)


def create_vpc(boto_session, cidr, name):
    ec2 = boto_session.resource('ec2')
    vpc = ec2.create_vpc(CidrBlock = cidr, 
                         TagSpecifications=[{'ResourceType': 'vpc', 'Tags': [{'Key': 'Name', 'Value': name}]}])
    vpc.wait_until_available()
    #default security group, open internal traffic
    for sg in vpc.security_groups.all():
        sg_tag = name + '-internal-securitygroup'
        ec2.SecurityGroup(sg.id).create_tags(Tags=[{'Key': 'Name','Value': sg_tag}])
    #default route table
    for rt in vpc.route_tables.all():
        rt_tag = name + '-route-table'
        ec2.RouteTable(rt.id).create_tags(Tags=[{'Key': 'Name','Value': rt_tag}])
        
    return vpc, sg, rt 
    
def create_subnet(vpc, cidr, az, name):
    subnet = vpc.create_subnet(AvailabilityZone = az,
                      CidrBlock = cidr,
                      TagSpecifications=[{'ResourceType': 'subnet', 'Tags': [{'Key': 'Name','Value': name}]}])

    return subnet

def create_internet_gateway(boto_session, vpc, name):
    ec2 = boto_session.resource('ec2')
    ig = ec2.create_internet_gateway(TagSpecifications=[{'ResourceType': 'internet-gateway', 'Tags': [{'Key': 'Name','Value': name}]}])
    vpc.attach_internet_gateway(InternetGatewayId=ig.id)

    return ig

def create_route_table(vpc, name):
    rt = vpc.create_route_table(TagSpecifications=[{'ResourceType': 'route-table', 'Tags': [{'Key': 'Name','Value': name}]}])

    return rt

def create_db_subnet_group(boto_session, db_identifier, subnet_ids):
    rds = boto_session.client('rds')
    dbsubnet = rds.create_db_subnet_group(DBSubnetGroupDescription='{db} subnet group'.format(db=db_identifier),
                                          DBSubnetGroupName='{db}-subnetgroup'.format(db=db_identifier),
                                          SubnetIds=[dbsubnet01.id,dbsubnet02.id],
                                          Tags=[{'Key': 'Name','Value': '{db}-subnetgroup'.format(db=db_identifier)}])
    
    return dbsubnet

def create_external_ec2_security_group(boto_session, vpc, name):
    ec2 = boto_session.resource('ec2')
    secgrp = ec2.create_security_group(GroupName=name,
                                   Description='basic security group for ec2',
                                   VpcId=vpc.id,
                                   TagSpecifications=[{'ResourceType': 'security-group', 'Tags': [{'Key': 'Name','Value': name}]}])
    ip22 = get_public_ip() + '/32'
    secgrp.authorize_ingress(GroupId=secgrp.id,
                             IpPermissions=[
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 80,
                                  'ToPort': 80,
                                  'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 443,
                                  'ToPort': 443,
                                  'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 22,
                                  'ToPort': 22,
                                  'IpRanges': [{'CidrIp': ip22}]}
                             ])

    return secgrp

def create_mysqldb_security_group(boto_session, vpc, name):
    ec2 = boto_session.resource('ec2')
    secgrp = ec2.create_security_group(GroupName=name,
                                   Description='basic security group for mysql db',
                                   VpcId=vpc.id,
                                   TagSpecifications=[{'ResourceType': 'security-group', 'Tags': [{'Key': 'Name','Value': name}]}])
    secgrp.authorize_ingress(GroupId=secgrp.id,
                             IpPermissions=[
                                 {'IpProtocol': 'tcp',
                                  'FromPort': 3306,
                                  'ToPort': 3306,
                                  'IpRanges': [{'CidrIp': '0.0.0.0/0'}]}
                             ])
    return secgrp

In [79]:
asm = boto3.Session(profile_name = 'mjn', region_name = 'us-west-2')

In [89]:
prj_vpc, internal_sg, route_tbl = create_vpc(asm, '213.213.48.0/24', 'tripoli-vpc')

In [90]:
dbsubnet01 = create_subnet(prj_vpc, '213.213.48.32/27', 'us-west-2b', 'tripoli-dbsubnet-01')
dbsubnet02 = create_subnet(prj_vpc, '213.213.48.64/27', 'us-west-2c', 'tripoli-dbsubnet-02')
publicsubnet = create_subnet(prj_vpc, '213.213.48.96/27', 'us-west-2a', 'tripoli-publicsubnet')

In [91]:
intgateway = create_internet_gateway(asm, prj_vpc, 'tripoli-ig')
route_tbl.create_route(DestinationCidrBlock='0.0.0.0/0',GatewayId=intgateway.id)

In [97]:
dbsubnetgrp = create_db_subnet_group(asm, 'tripoli-db', [dbsubnet01.id, dbsubnet02.id])

In [101]:
rds = asm.client('rds')

In [104]:
mysqldb_rds = rds.create_db_instance(DBName='tripoliwordpress',
                       DBInstanceIdentifier='tripoli-db',
                       AllocatedStorage=20,
                       DBInstanceClass='db.t2.micro',
                       Engine='mysql',
                       MasterUsername='tripoliadmin',
                       MasterUserPassword='Tripoli2021!',
                       StorageType='gp2',
                       StorageEncrypted=False,
                       AutoMinorVersionUpgrade=True,
                       MultiAZ=False,
                       DBSubnetGroupName=dbsubnetgrp['DBSubnetGroup']['DBSubnetGroupName'],
                       PubliclyAccessible=False,
                       VpcSecurityGroupIds=[internal_sg.id],
                       Tags=[{'Key': 'Name', 'Value': 'tripoli-db'}])

{'DBInstance': {'DBInstanceIdentifier': 'tripoli-db',
  'DBInstanceClass': 'db.t2.micro',
  'Engine': 'mysql',
  'DBInstanceStatus': 'creating',
  'MasterUsername': 'tripoliadmin',
  'DBName': 'tripoliwordpress',
  'AllocatedStorage': 20,
  'PreferredBackupWindow': '10:56-11:26',
  'BackupRetentionPeriod': 1,
  'DBSecurityGroups': [],
  'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-0b71f6232d2ae32ee',
    'Status': 'active'}],
  'DBParameterGroups': [{'DBParameterGroupName': 'default.mysql8.0',
    'ParameterApplyStatus': 'in-sync'}],
  'DBSubnetGroup': {'DBSubnetGroupName': 'tripoli-db-subnetgroup',
   'DBSubnetGroupDescription': 'tripoli-db subnet group',
   'VpcId': 'vpc-01ee38ee20c0a19a1',
   'SubnetGroupStatus': 'Complete',
   'Subnets': [{'SubnetIdentifier': 'subnet-07bfffa9e3ed0fcd2',
     'SubnetAvailabilityZone': {'Name': 'us-west-2c'},
     'SubnetOutpost': {},
     'SubnetStatus': 'Active'},
    {'SubnetIdentifier': 'subnet-02c0e7a73da4c028a',
     'SubnetAvailabilityZone

In [107]:
ec2secgrp = create_external_ec2_security_group(asm, prj_vpc, 'tripoli-ec2-securitygroup')

In [108]:
ec2secgrp.id

'sg-03cbc45141e98b963'

In [111]:
ec2 = asm.resource('ec2')

In [121]:
priv_key = ec2.create_key_pair(KeyName='tripoli_ec2_key')

In [130]:
tripoli_ec2 = ec2.create_instances(ImageId='ami-00f9f4069d04c0c6e',
                     InstanceType='t2.micro',
                     MaxCount=1,
                     MinCount=1,
                     SubnetId=publicsubnet.id,
                     BlockDeviceMappings=[{'DeviceName': '/dev/xvda',
                                           'VirtualName': 'tripoli-storage',
                                           'Ebs': {'DeleteOnTermination': True,
                                                   'VolumeSize': 30}
                                          }],
                     TagSpecifications=[{'ResourceType': 'instance', 'Tags': [{'Key': 'Name','Value': 'tripoli-ec2'}]}],
                     SecurityGroupIds=[ec2secgrp.id],
                     KeyName=priv_key.name)